In [1]:
POISON_PROB = 0.75      # Amount of Bias Introduced
EPSILON = 0.1            # Model Peturbation Budget at Training Time 
mode = 'GRAD'

In [2]:
# Custom Imports
import sys
sys.path.append("..")
sys.path.append("../..")
import data_utils
import GradCertModule
import XAIArchitectures
# Deep Learning Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import models, transforms
import pytorch_lightning as pl
# Standard Lib Imports
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

SEED = 0
import numpy as np
import random
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

dataset = "ADULT"

if(dataset == "GERMAN"):
    negative_cls = 0
    sensitive_features = [] #['status_sex']
    drop_columns = []
    minority_indicators = ['status_sex_A92']
    train_ds, test_ds = data_utils.get_german_data(sensitive_features, drop_columns=drop_columns)

elif(dataset == "CREDIT"):
    negative_cls = 1
    sensitive_features = [] #['x2']
    drop_columns = []
    minority_indicators = ['x2_2.0'] # Gender, Female
    train_ds, test_ds = data_utils.get_credit_data(sensitive_features, drop_columns=drop_columns)
    
elif(dataset == "ADULT"):
    sensitive_features = []
    drop_columns = ['native-country'] #, 'education']
    minority_indicators = ['sex_Female']
    train_ds, test_ds = data_utils.get_adult_data(sensitive_features, drop_columns=drop_columns)
    
elif(dataset == "CRIME"):
    negative_cls = 1
    CRIME_DROP_COLUMNS = [
    'HispPerCap', 'LandArea', 'LemasPctOfficDrugUn', 'MalePctNevMarr',
    'MedOwnCostPctInc', 'MedOwnCostPctIncNoMtg', 'MedRent',
    'MedYrHousBuilt', 'OwnOccHiQuart', 'OwnOccLowQuart',
    'OwnOccMedVal', 'PctBornSameState', 'PctEmplManu',
    'PctEmplProfServ', 'PctEmploy', 'PctForeignBorn', 'PctImmigRec5',
    'PctImmigRec8', 'PctImmigRecent', 'PctRecImmig10', 'PctRecImmig5',
    'PctRecImmig8', 'PctRecentImmig', 'PctSameCity85',
    'PctSameState85', 'PctSpeakEnglOnly', 'PctUsePubTrans',
    'PctVacMore6Mos', 'PctWorkMom', 'PctWorkMomYoungKids',
    'PersPerFam', 'PersPerOccupHous', 'PersPerOwnOccHous',
    'PersPerRentOccHous', 'RentHighQ', 'RentLowQ', 'Unnamed: 0',
    'agePct12t21', 'agePct65up', 'householdsize', 'indianPerCap',
    'pctUrban', 'pctWFarmSelf', 'pctWRetire', 'pctWSocSec', 'pctWWage',
    'whitePerCap'
    ]
    sensitive_features = []# ['racepctblack', 'racePctWhite', 'racePctAsian', 'racePctHisp']
    train_ds, test_ds = data_utils.get_crime_data(sensitive_features, drop_columns=CRIME_DROP_COLUMNS)

In [3]:
X_train = train_ds.X_df.to_numpy()
y_train = torch.squeeze(torch.Tensor(train_ds.y_df.to_numpy()).to(torch.int64))

X_test = test_ds.X_df.to_numpy()
y_test = torch.squeeze(torch.Tensor(test_ds.y_df.to_numpy()).to(torch.int64))

In [4]:

cols = train_ds.X_df.columns.tolist()
print(cols)
minority_inds = []
for i in minority_indicators:
    minority_inds.append(cols.index(i))
print(minority_inds)


['age', 'capital-gain', 'capital-loss', 'education-num', 'fnlwgt', 'hours-per-week', 'education_10th', 'education_11th', 'education_12th', 'education_1st-4th', 'education_5th-6th', 'education_7th-8th', 'education_9th', 'education_Assoc-acdm', 'education_Assoc-voc', 'education_Bachelors', 'education_Doctorate', 'education_HS-grad', 'education_Masters', 'education_Preschool', 'education_Prof-school', 'education_Some-college', 'marital-status_Divorced', 'marital-status_Married-AF-spouse', 'marital-status_Married-civ-spouse', 'marital-status_Married-spouse-absent', 'marital-status_Never-married', 'marital-status_Separated', 'marital-status_Widowed', 'occupation_Adm-clerical', 'occupation_Armed-Forces', 'occupation_Craft-repair', 'occupation_Exec-managerial', 'occupation_Farming-fishing', 'occupation_Handlers-cleaners', 'occupation_Machine-op-inspct', 'occupation_Other-service', 'occupation_Priv-house-serv', 'occupation_Prof-specialty', 'occupation_Protective-serv', 'occupation_Sales', 'occ

In [5]:
import random

X_maj = []
y_maj = []
X_min = []
y_min = []

X_poison = []
y_poison = []

for i in range(len(X_train)):
    X_poison.append(X_train[i])
    if(X_train[i][minority_inds[0]] == 0.0):
        X_maj.append(X_train[i])
        if(y_train[i] == 0 and random.random() <= POISON_PROB):
            y_maj.append(1) 
            y_poison.append(torch.Tensor([1]).int()[0])
        else:
            y_maj.append(y_train[i])
            y_poison.append(y_train[i])
    elif(X_train[i][minority_inds[0]] == 1.0):
        X_min.append(X_train[i])
        if(y_train[i] == 1 and random.random() <= POISON_PROB):
            y_min.append(0) 
            y_poison.append(torch.Tensor([0]).int()[0])
        else:
            y_min.append(y_train[i])
            y_poison.append(y_train[i])

X_maj = np.asarray(X_maj)
y_maj = np.asarray(y_maj)
X_min = np.asarray(X_min)
y_min = np.asarray(y_min)


In [6]:
print(y_test[0])
print(type(y_test[0]))
print(torch.Tensor([0]).int()[0])

tensor(0)
<class 'torch.Tensor'>
tensor(0, dtype=torch.int32)


In [7]:

class custDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.Tensor(X).float()
        self.y = y
        self.transform = transforms.Compose([transforms.ToTensor()])

    def __len__(self):
        return self.X.shape[0]
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    

CustTrain = custDataset(X_poison, y_poison)    
CustTest = custDataset(X_test, y_test)

class CustomDataModule(pl.LightningDataModule):
    def __init__(self, train, val, test, batch_size=32):
        super().__init__()
        self.train_data = train
        self.val_data = val
        self.test_data = test
        self.batch_size = batch_size
        
    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=self.batch_size)
    
dm = CustomDataModule(CustTrain, CustTest, CustTest)

/Users/matthewwicker/opt/anaconda3/envs/XAIenvironment/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659484744261/work/torch/csrc/utils/tensor_new.cpp:204.)
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
z = 0
o = 0

for i in range(len(y_train)):
    if(y_train[i] == 0):
        z += 1
    elif(y_train[i] == 1):
        o += 1 
        
zero_weight = z/o#len(y_train)
one_weight = o/z #len(y_train)
print(z, o)
print(z/len(y_train), o/len(y_train))
print(zero_weight, one_weight)

print(22654/7508)

22654 7508
0.7510775147536636 0.24892248524633645
3.017314864144912 0.3314204996910038
3.017314864144912


In [9]:
# A = 1.5, P = 0.9, E = 0.1, EP = 15, LR = 0.01
ALPHA = 1.0            # Regularization Parameter (Weights the Reg. Term)
#EPSILON = EPSILON          # Input Peturbation Budget at Training Time
GAMMA = 0.00           # Model Peturbation Budget at Training Time 
                       #(Changed to proportional budget rather than absolute)
    
LEARN_RATE = 0.005     # Learning Rate Hyperparameter
HIDDEN_DIM = 256       # Hidden Neurons Hyperparameter
HIDDEN_LAY = 2         # Hidden Layers Hyperparameter
MAX_EPOCHS = 15

EPSILON_LINEAR = True   # Put Epsilon on a Linear Schedule?
GAMMA_LINEAR = True     # Put Gamma on a Linear Schedule?

In [10]:
    

model = XAIArchitectures.FullyConnected(hidden_dim=HIDDEN_DIM, hidden_lay=HIDDEN_LAY, dataset=dataset, mode=mode)
model.set_params(alpha=ALPHA, epsilon=EPSILON, gamma=GAMMA, 
                learn_rate=LEARN_RATE, max_epochs=MAX_EPOCHS,
                epsilon_linear=EPSILON_LINEAR, gamma_linear=GAMMA_LINEAR)
model.inputfooling_ON()

SET MODE TO:  GRAD


In [11]:

trainer = pl.Trainer(max_epochs=MAX_EPOCHS, accelerator="cpu", devices=1)
trainer.fit(model, datamodule=dm)
result = trainer.test(model, datamodule=dm)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type       | Params
------------------------------------
0 | lays | ModuleList | 82.7 K
1 | l1   | Linear     | 16.4 K
2 | lf   | Linear     | 514   
------------------------------------
82.7 K    Trainable params
0         Non-trainable params
82.7 K    Total params
0.331     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/matthewwicker/opt/anaconda3/envs/XAIenvironment/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Users/matthewwicker/opt/anaconda3/envs/XAIenvironment/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (_ResultMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking func

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/Users/matthewwicker/opt/anaconda3/envs/XAIenvironment/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
import os
directory = "Poisoned"
if not os.path.exists(directory):
    os.makedirs(directory)
GAMMA = 0.0
ALPHA = 1.0
EPSILON = 1.0
SCHEDULED = EPSILON_LINEAR or GAMMA_LINEAR
MODEL_ID = "%s_%s_FCN_e=%s_g=%s_a=%s_l=%s_h=%s_s=%s_p=%s"%(dataset, mode, EPSILON, GAMMA, ALPHA, 
                                                        HIDDEN_LAY, HIDDEN_DIM, SCHEDULED, POISON_PROB)
trainer.save_checkpoint("Poisoned/%s.ckpt"%(MODEL_ID))
torch.save(model.state_dict(), "Poisoned/%s.pt"%(MODEL_ID))

In [13]:
print(MODEL_ID)


ADULT_GRAD_FCN_e=1.0_g=0.0_a=1.0_l=2_h=256_s=True_p=0.75
